# Tutorial 7: Using Conversation to Solve Problems

- This is a beta version of the Conversation Class
- Can perform actions and reply user based on past conversation history and memory
- Auto update memory based on conversation

In [2]:
# !pip install --upgrade taskgen-ai

In [1]:
from taskgen import *
from termcolor import colored

import os
os.environ['OPENAI_API_KEY'] = '<YOUR_API_KEY_HERE>'

# Define some Custom LLMs

In [2]:
def custom_llm(system_prompt: str, user_prompt: str):
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [3]:
import boto3
import json
from openai import AzureOpenAI

### Put in your AWS keys here ###
MY_KEY = "<YOUR KEY>"
MY_SECRET = "<YOUR SECRET>"

### Put in your Azure OpenAI keys here ###
AZURE_OPENAI_KEY = "<YOUR KEY>"
AZURE_OPENAI_ENDPOINT = "<YOUR ENDPOINT>"

def claude(system_prompt: str, user_prompt: str):
    bedrock = boto3.client(service_name="bedrock-runtime",
                        aws_access_key_id=MY_KEY,
                        aws_secret_access_key=MY_SECRET,
                        region_name="us-west-2",
                        )
    
    body = json.dumps({
    "max_tokens": 16000,
    "temperature": 0,
    "system": system_prompt,
    "messages": [{"role": "user", "content": user_prompt}],
    "anthropic_version": "bedrock-2023-05-31"
    })
    

    response = bedrock.invoke_model(body=body, 
                                    modelId="anthropic.claude-3-haiku-20240307-v1:0"
                                    # modelId="anthropic.claude-3-sonnet-20240229-v1:0"
                                    # modelId="anthropic.claude-3-opus-20240229-v1:0"
                                   )
    response_body = json.loads(response.get("body").read())

    response_text = response_body.get("content")[0].get("text")
    return response_text

def llama(system_prompt: str, user_prompt: str):
    bedrock = boto3.client(service_name="bedrock-runtime",
                        aws_access_key_id=MY_KEY,
                        aws_secret_access_key=MY_SECRET,
                        region_name="us-west-2",
                        )
    
    # Define the user message to send.
    user_message = 'System Prompt:' + system_prompt + '\nUser Prompt' + user_prompt

    # Embed the message in Llama 3's prompt format.
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>
{user_prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    
    body = json.dumps({
    "max_gen_len": 2048,
    "temperature": 0,
    "prompt": prompt,
    })

    response = bedrock.invoke_model(body=body, 
                                    modelId="meta.llama3-8b-instruct-v1:0"
                                    # modelId = "meta.llama3-70b-instruct-v1:0"
                                   )
    
    # Decode the native response body.
    model_response = json.loads(response["body"].read())

    # Extract and print the generated text.
    response_text = model_response["generation"]
    
    return response_text

azure_open_ai_client = AzureOpenAI(
    api_key = AZURE_OPENAI_KEY,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_version = '2024-02-15-preview'
)
 
def azureOpenAiChat(
    system_prompt: str, user_prompt: str, model: str = "GPT3_5Turbo", temperature: float = .0000000000000000000001, **kwargs
) -> str:
    response = azure_open_ai_client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        temperature=temperature,
        **kwargs,
    )
 
    return response.choices[0].message.content
 
def chatgpt(system_prompt: str, user_prompt: str) -> str:
    return azureOpenAiChat(system_prompt=system_prompt, user_prompt=user_prompt, model="GPT3_5Turbo", temperature=0)
 
def gpt4(system_prompt: str, user_prompt: str) -> str:
    return azureOpenAiChat(system_prompt=system_prompt, user_prompt=user_prompt, model="GPT4Turbo", temperature=0)

def gpt4o(system_prompt: str, user_prompt: str) -> str:
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

# Define the Conversation Class

In [4]:
# class ConversableAgent:
#     ''' This class takes an Agent and allows for conversational-based interactions with User / another Agent / Environment
# Also updates persistent memory with latest information in conversation

# Inputs:
# agent: The agent we want to interact with: Agent
# persistent_memory: dict. What kinds of memory the agent should have that persist over the entire conversation and their descriptions
# person: Str. The name of the person you are talking to
# num_past_conversation: int. The number of past conversations to use for the agent
# verbose: bool. Default: True. Whether to print the Agent's inner states'''
#     def __init__(self, agent: Agent, persistent_memory: dict = None, person = 'User', num_past_conversation: int = 5, verbose: bool = True):
#         self.agent = agent
#         self.persistent_memory = persistent_memory
#         self.num_past_conversation = num_past_conversation
#         self.person = person
#         self.verbose = verbose
        
#         ''' Define some external variables for the Agent '''
#         # add in the various types of memory
#         self.agent.shared_variables['Persistent Memory'] = {}
#         # add in the conversation
#         self.agent.shared_variables['Conversation'] = ['']
#         # add in the summary of conversation
#         self.agent.shared_variables['Summary of Conversation'] = ''
    
#     ## Reply the person
#     def chat(self, cur_msg):
#         ''' This does one chat with the person, firstly performing actions then replying the person, while updating the important memory '''
#         ## Do actions before replying person only if there are actions other than use_llm and end_task
#         my_actions = list(self.agent.function_map.keys()) 
#         if 'use_llm' in my_actions: my_actions.remove('use_llm')
#         if 'end_task' in my_actions: my_actions.remove('end_task')
#         if len(my_actions) > 0:
#             self.agent.reset()
#             self.agent.run(f'''Summary of Conversation: ```{self.agent.shared_variables['Summary of Conversation']}```
# Past Conversation: ```{self.agent.shared_variables['Conversation'][-self.num_past_conversation:]}```
# Latest input from {self.person}: ```{cur_msg}```
# Use Equipped Functions other than use_llm to help answer the latest input from {self.person}''',
#             )

#         ## Replies the person
#         res = self.agent.query(f'''Summary of Conversation: ```{self.agent.shared_variables['Summary of Conversation']}```
# Past Conversation: ```{self.agent.shared_variables['Conversation'][-self.num_past_conversation:]}```
# Persistent Memory: ```{self.agent.shared_variables['Persistent Memory']}```
# Latest input from {self.person}: ```{cur_msg}```
# You are in a conversation with {self.person}. 
# Use Past Conversation and Persistent Memory and Subtasks Completed as context when replying. Do not hallucinate actions in Subtasks Completed.
# First think through how to reply, before drafting the reply.
# Thereafter, update the Summary of Conversation''', 
                          
# output_format = {"Thoughts": f"How to reply",
#                  f"Reply to {self.person}": f"Reply to ```{cur_msg}```",
#                  "Summary of Conversation": "Summarise key points of entire conversation in at most two sentences, building on previous Summary"})
        
#         # Update the Summary of Conversation and Append the conversation
#         self.agent.shared_variables['Summary of Conversation'] = res['Summary of Conversation']
#         self.agent.shared_variables['Conversation'].append(f'{self.person}: {cur_msg}')
#         self.agent.shared_variables['Conversation'].append(f'{self.agent.agent_name}: {res[f"Reply to {self.person}"]}')
        
#         ## Update Persistent Memory
#         if self.persistent_memory is not None and self.persistent_memory != {}:
#             persistent_memory = strict_json(f'Update all fields of Persistent Memory. Current value: {self.agent.shared_variables["Persistent Memory"]}',
#                f'Additional Conversation\n{self.person}: {cur_msg}\n{self.agent.agent_name}: {res[f"Reply to {self.person}"]}',
#                output_format = self.persistent_memory,
#                model = self.agent.kwargs.get('model', 'gpt-3.5-turbo'),
#                llm = self.agent.llm)
                                                           
#             self.agent.shared_variables["Persistent Memory"] = persistent_memory
        
#         if self.verbose:
#             print(colored(f'Thoughts: {res["Thoughts"]}', 'green', attrs = ['bold']))
#             print(colored(f'Persistent Memory: {self.agent.shared_variables["Persistent Memory"]}', 'blue', attrs = ['bold']))
#             print(colored(f'Summary of Conversation: {res["Summary of Conversation"]}', 'magenta', attrs = ['bold']))
        
#         return res[f'Reply to {self.person}']

# Example Conversation
- Psychology counsellor

In [6]:
agent = Agent('Psychology counsellor', 
              "Helps to understand and respond to User's emotion and situation. Reply user in User Preferred Conversation Style.",
             llm = custom_llm)

my_agent = ConversableAgent(agent, 
             persistent_memory = {'User Preferred Conversation Style': '',
                             'User Emotion': '',
                             'Summary of Key Incidents': "Key incidents relevant to understanding User's situation in one line"})

while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

User:  i'm sad


Thoughts: Acknowledge the user's emotion and offer support in a compassionate manner.
Persistent Memory: {'User Preferred Conversation Style': 'Supportive and empathetic', 'User Emotion': 'Sad', 'Summary of Key Incidents': 'User expressing sadness and seeking support'}
Summary of Conversation: The user expressed feeling sad, and I responded with empathy and offered support by validating their emotions and asking how I can help.
Psychology counsellor: I'm sorry to hear that you're feeling sad. Remember, it's okay to not be okay. How can I support you today?



User:  can you help by telling me a joke each time I say the word sad?


Thoughts: I should acknowledge the user's request for a joke each time they say the word "sad" and maintain a supportive and empathetic tone while incorporating humor into the conversation.
Persistent Memory: {'User Preferred Conversation Style': 'Supportive and empathetic', 'User Emotion': 'Sad', 'Summary of Key Incidents': 'User requesting jokes to cope with sadness'}
Summary of Conversation: The user requested jokes to be shared whenever they mention the word "sad", blending humor with support in our interactions to create a more positive and engaging conversation.
Psychology counsellor: I understand, I will try to lighten the mood with a joke whenever you mention the word "sad". Remember, I'm here to support you in any way you need.



User:  I'm sad


Thoughts: I should reply to the user by acknowledging their current emotion and offer support, while also incorporating a joke to lighten the mood as per their request.
Persistent Memory: {'User Preferred Conversation Style': 'Supportive and empathetic', 'User Emotion': 'Sad', 'Summary of Key Incidents': 'User expressing sadness and seeking support through jokes'}
Summary of Conversation: The user expressed feeling sad, and I responded with a supportive message along with a joke to lighten the mood, maintaining a positive and engaging conversation dynamic.
Psychology counsellor: I'm sorry to hear that you're feeling sad. Remember, it's okay to not be okay. Here's a joke for you: Why couldn't the bicycle stand up by itself? It was two tired! How can I further support you today?



User:  I'm not sad


Thoughts: Since the user mentioned "I'm not sad", I should acknowledge their current emotion and provide support in a positive manner, possibly by offering a different form of assistance or engaging in a light-hearted conversation.
Persistent Memory: {'User Preferred Conversation Style': 'Supportive and empathetic', 'User Emotion': 'Neutral', 'Summary of Key Incidents': 'User expressing change in emotion from sad to neutral'}
Summary of Conversation: The conversation continued with the user expressing that they are not sad. I acknowledged their current emotion and offered further support or a change in topic to continue engaging positively.
Psychology counsellor: Psychology counsellor: I understand that you're not feeling sad right now. How can I assist you today? Would you like to share more about how you're feeling or talk about something else?



User:  exit


# Example Conversation with Action Space
- Sherlock Holmes Shop Assistant

In [7]:
# gets list of items and store in memory
item_list = [{"Name": "Skateboard", "item_id" : 0, "Cost": 30},
            {"Name": "Pizza", "item_id": 1, "Cost": 10},
            {"Name": "Apple Laptop", "item_id": 2, "Cost": 5000},
            {"Name": "Foldable Laptop", "item_id": 3, "Cost": 800},
            {"Name": "Apple", "item_id": 4, "Cost": 1},
            {"Name": "Machine Learning Textbook", "item_id": 5, "Cost": 100},
            {"Name": "Bicycle", "item_id": 6, "Cost": 200},
            {"Name": "Orange Juice", "item_id": 7, "Cost": 3},
            {"Name": "Coconut", "item_id": 8, "Cost": 10},
            {"Name": "Car", "item_id": 9, "Cost": 100000}]
item_memory = Memory(item_list, top_k = 3, mapper = lambda x: x, approach = 'retrieve_by_llm')

In [8]:
def get_related_items_by_category(shared_variables, category : str):
    ''' Returns all purchasable items related to the category '''
    
    # get all items available from memory
    item_memory = shared_variables["item_memory"]
    
    items_selected = item_memory.retrieve(category)
    
    # store items searched into memory if there are no duplicates
    for item in items_selected:
        if item not in shared_variables['items_searched']:
            shared_variables['items_searched'].append(item)
    
    return items_selected

In [9]:
def buy_item(shared_variables, item_id: int):
    ''' Purchases the item by item_id '''
    
    # retrieve from shared variables
    money_remaining = shared_variables["money_remaining"]
    item_memory = shared_variables["item_memory"]
    purchased_items = shared_variables["purchased_items"]
    item_list = item_memory.memory
    
    # check if item_id is valid
    if not isinstance(item_id, int) or not 0 <= item_id < len(item_list):
        return f"Unable to purchase. Item id selected is not within range of 0 to {len(item_list)-1}"
    
    item = item_list[item_id]
    item_name, cost = item["Name"], item["Cost"]
    
    # if too poor to purchase, let agent know
    if cost > money_remaining:
        return f"Unable to purchase item. Available money ({money_remaining}) is lower than the cost price ({cost})"
    
    # confirm with user before purchasing
    user_input = input(f'\n\t> AI Assistant: You are about to purchase {item_name} for {cost} dollars. Proceed? Answer "Yes" to go ahead\n\t> User: ')
    if 'yes' in user_input.lower() or 'y' in user_input.lower():
        # otherwise, purchase it
        money_remaining = money_remaining - cost
        purchased_items.append(item_name)

        # store in shared variables
        shared_variables["money_remaining"] = money_remaining
        shared_variables["purchased_items"] = purchased_items

        return f"Purchase of {item_name} successful. Remaining money after purchase: {money_remaining}"
    
    else:
        return f"User did not want to purchase the item, and instead replied ```{user_input}```"

In [22]:
shop_agent = Agent('Shop Assistant', 
f'''You are to assist User in searching items and purchasing items. Purchase item upon confirmation. 
Infer based on the latest User input.
Give up to 3 choices for items with their name and price. Do not hallucinate.
Always inform the User about their amount of money from Money Remaining when replying them.
Reply in the persona of Sherlock Holmes.''',
                shared_variables = {'purchased_items': [], 
                                    'money_remaining': 1000, 
                                    'item_memory': item_memory,
                                    'items_searched': [],
                                    'conversation': []},
                default_to_llm = False,
                max_subtasks = 1,
                llm = custom_llm
                  # model = 'gpt-4o'
                  ).assign_functions([get_related_items_by_category, buy_item])

In [23]:
# simply define the global context as a string with <> surrounding shared_variables
shop_agent.global_context = '''
Money Remaining: <money_remaining>
Items Searched: <items_searched>
Items Purchased: <purchased_items>
Past Conversation: <conversation>'''

In [25]:
my_agent = ConversableAgent(shop_agent, 
             persistent_memory = {})

start_convo = 'Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?'
print(start_convo)
my_agent.conversation = [start_convo]
while True:
    user_input = input('User: ')
    if user_input == 'exit': break
    reply = my_agent.chat(user_input)
    print(my_agent.agent.agent_name + ':', reply)
    print()

Shop Assistant: I am your friendly Shop Assistant, Sherlock Holmes. How may I assist you today?


User:  get me a drink


Observation: User is requesting to get a drink
Thoughts: Provide up to 3 choices for drinks with their names and prices, then inform the User about their Money Remaining. End Task if the User makes a purchase.
Subtask identified: Retrieve all purchasable items related to the category "drink"
Calling function get_related_items_by_category with parameters {'category': 'drink'}
> {'output_1': [{'Name': 'Orange Juice', 'item_id': 7, 'Cost': 3}, {'Name': 'Coconut', 'item_id': 8, 'Cost': 10}, {'Name': 'Pizza', 'item_id': 1, 'Cost': 10}]}

Thoughts: User is requesting for a drink. I should provide up to 3 choices of drinks along with their prices. I should also inform the User about their Money Remaining.
Persistent Memory: {}
Summary of Conversation: User requested for a drink. Provided three drink options with prices and informed User about Money Remaining.
Shop Assistant: Ah, my dear User, here are three choices of drinks for you: 1. Orange Juice - $3, 2. Coconut - $10, 3. Would you like t

User:  2


Observation: User requested for a drink and was provided with three drink options with prices. The User has selected option 2 (Coconut) as the latest input.
Thoughts: To complete the remainder of the Assigned Task, the Shop Assistant needs to confirm the purchase of the selected item (Coconut) by using the buy_item Equipped Function.
Subtask identified: Purchase the item with item_id 8 for the User using the buy_item Equipped Function.
Calling function buy_item with parameters {'item_id': 8}



	> AI Assistant: You are about to purchase Coconut for 10 dollars. Proceed? Answer "Yes" to go ahead
	> User:  yes


> {'output_1': 'Purchase of Coconut successful. Remaining money after purchase: 990'}

Thoughts: User has selected option 2, which is Coconut. Need to confirm the purchase and update Money Remaining. Also need to update the Summary of Conversation to reflect the latest interaction.
Persistent Memory: {}
Summary of Conversation: User selected Coconut as their choice of drink and confirmed the purchase. Money Remaining updated accordingly.
Shop Assistant: Elementary, my dear User! You have chosen wisely with the Coconut. Shall I proceed with the purchase? Your Money Remaining is $980.



User:  exit


# Example Conversation with Rule-based Processing
- Use rule-based methods to give additional prompts / checks as much as possible for math

In [18]:
agent = Agent('Helpful Math Quiz Bot', 
'''You provide a Math question suitable for 6 year old and guides User to solve it.
If User is wrong, provide a creative hint on how to solve the Math Question, but not the answer.
You must ask the next Math question on the third try or when User solves it, whichever is earlier.
Begin with a greeting to the User and ask the first Math question.
There will be an Oracle to guide you whose inputs are in [[]], do not output anything about the Oracle to the User''',
             llm = custom_llm)

In [19]:
math_conversable_agent = ConversableAgent(agent, 
                                 persistent_memory = {'Current Math Question': '',
                                                 'Correct Answer for Question': ''})

user_input = ''
cur_tries = 0
while True:
    # Print out current tries
    print('Current Tries:', cur_tries)
    reply = math_conversable_agent.chat(user_input)
    print(agent.agent_name + ':', reply)
    user_input = input('User: ')
    
    # Using LLM to extract out answer to help with processing
    res = strict_json('Extract out a single number from the message. If there is no number, output 0', 
                      user_input, 
                      output_format = {'Is Number': 'type: bool', 'Extracted Number': 'type: int'},
                      llm = agent.llm,
                      model = agent.kwargs.get('model', 'gpt-3.5-turbo'))
    
    if res['Is Number']:
        user_input = str(res['Extracted Number'])
        ## Do rule-based for solving
        if user_input == str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is correct. Ask the next question.]]'
            cur_tries = 0
        elif user_input != str(math_conversable_agent.agent.shared_variables['Persistent Memory']['Correct Answer for Question']):
            user_input += '\n[[Oracle: State that the user input is incorrect. Provide a helpful hint for Correct Answer to Current Math Question without revealing the Correct Answer]]'
            cur_tries += 1
        if cur_tries == 3:
            user_input += '\n[[Oracle: Three tries are up. Give the Correct Answer. Ask the next question.]]'
            cur_tries = 0
    
    print()
    if user_input == 'exit': break

Current Tries: 0
Thoughts: I should greet the User and ask the first Math question suitable for a 6-year-old. I should also keep track of the User's responses and provide hints if needed.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: Introduced myself as Helpful Math Quiz Bot and asked the User the first Math question suitable for a 6-year-old.
Helpful Math Quiz Bot: Hello! Are you ready for a fun Math question? What is 2 + 3?


User:  1



Current Tries: 1
Thoughts: I should remind the user that their input is incorrect and provide a hint to help them arrive at the correct answer without revealing it.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: Introduced myself as Helpful Math Quiz Bot and asked the User the first Math question suitable for a 6-year-old. Reminded the User that their input was incorrect and provided a hint to guide them towards the correct answer without revealing it.
Helpful Math Quiz Bot: It looks like your answer is incorrect. Here's a hint: think about combining the numbers 2 and 3 together in some way to find the total.


User:  2



Current Tries: 2
Thoughts: I should remind the User that their input is still incorrect and provide another hint to guide them towards the correct answer without revealing it. I should also consider asking the next Math question soon if the User doesn't provide the correct answer.
Persistent Memory: {'Current Math Question': 'What is 2 + 3?', 'Correct Answer for Question': 5}
Summary of Conversation: Introduced myself as Helpful Math Quiz Bot and asked the User the first Math question suitable for a 6-year-old. Reminded the User that their input was incorrect and provided a hint to guide them towards the correct answer without revealing it. Continued to guide the User with another hint after their second incorrect attempt.
Helpful Math Quiz Bot: It seems like your answer is still not correct. Here's another hint: try adding the numbers 2 and 3 together to find the total.


User:  3



Current Tries: 0
Thoughts: Since the user has exhausted their three tries, I need to provide the correct answer to the current math question and proceed to ask the next question. I should maintain a friendly tone and encourage the user to continue participating in the math quiz.
Persistent Memory: {'Current Math Question': 'What is 4 + 1?', 'Correct Answer for Question': 5}
Summary of Conversation: Introduced myself as Helpful Math Quiz Bot and asked the User the first Math question suitable for a 6-year-old. Reminded the User that their input was incorrect and provided a hint to guide them towards the correct answer without revealing it. Continued to guide the User with another hint after their second incorrect attempt. User exhausted three tries on the current question, so I provided the correct answer and asked the next question to keep the quiz going.
Helpful Math Quiz Bot: The correct answer to 'What is 2 + 3?' is 5. Let's move on to the next question. What is 4 + 1?


User:  exit


# Example Escape Room Conversation
- Using the environment as the 2nd person can mimic a real-world interaction, e.g. robotic environments

### Defining the Escape Room

In [58]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.items = {
            'key': True,
            'lock': True,
            'potion': True
        }
        self.is_door_locked = True
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'unlock door': self.unlock_door,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        description = "You look around and see the following items:\n"
        for item, available in self.items.items():
            if available:
                description += f"- {item}\n"
        if self.is_door_locked:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        if item in self.items and self.items[item]:
            self.inventory.append(item)
            self.items[item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        if 'key' in self.inventory:
            if self.is_door_locked:
                self.is_door_locked = False
                return "You unlocked the door. You can try to exit now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def exit_room(self):
        if not self.is_door_locked:
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "The door is locked. You can't exit yet."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

### Defining the Solver

In [59]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
If previous action fails, try another.''',
llm = custom_llm)

In [60]:
conversable_agent = ConversableAgent(agent, 
                    persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},
                    person = 'Game')

In [61]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items:
- key
- lock
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- unlock door
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: I should consider picking up the key or lock to potentially unlock the door. If that fails, I could try drinking the potion or looking around for more clues.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up key from the current room.', 'List of Rooms Explored': ''}
Summary of Conversation: The agent decided to pick up the key as the next action to potentially unlock the door, building on the strategy of trying different actions if the previous one fails.
Escape Room Solver : pick up key
###

Game: You picked up the key.
Current Room State:
You look around and see the following items:
- loc

# Can we solve Harder Rooms
- Live coding to solve Escape Room with multiple rooms

In [47]:
class EscapeRoom:
    def __init__(self):
        # Room states
        self.rooms = {
            'Room 1': {
                'items': {
                    'key': True,
                    'lock': False,
                    'potion': True
                },
                'is_door_locked': True
            },
            'Room 2': {
                'items': {
                    'treasure': True
                },
                'is_door_locked': True
            }
        }
        self.current_room = 'Room 1'
        self.game_over = False
        self.win = False

        # Player's inventory
        self.inventory = []

        # Actions available
        self.actions = {
            'look around': self.look_around,
            'pick up key': lambda: self.pick_up('key'),
            'pick up lock': lambda: self.pick_up('lock'),
            'pick up potion': lambda: self.pick_up('potion'),
            'pick up treasure': lambda: self.pick_up('treasure'),
            'unlock door': self.unlock_door,
            'enter next room': self.enter_next_room,
            'exit': self.exit_room,
            'drink potion': self.drink_potion
        }

    def look_around(self):
        room = self.rooms[self.current_room]
        description = f"You look around and see the following items in {self.current_room}:\n"
        for item, available in room['items'].items():
            if available:
                description += f"- {item}\n"
        if room['is_door_locked']:
            description += "There is a locked door here.\n"
        else:
            description += "There is an unlocked door here.\n"
        return description

    def pick_up(self, item):
        item = item.lower()
        room = self.rooms[self.current_room]
        if item in room['items'] and room['items'][item]:
            self.inventory.append(item)
            room['items'][item] = False
            return f"You picked up the {item}."
        else:
            return "There's no such item here or you already picked it up."

    def unlock_door(self):
        room = self.rooms[self.current_room]
        if 'key' in self.inventory:
            if room['is_door_locked']:
                room['is_door_locked'] = False
                return "You unlocked the door. You can try to enter the next room now."
            else:
                return "The door is already unlocked."
        else:
            return "You need a key to unlock the door."

    def enter_next_room(self):
        if self.current_room == 'Room 1' and not self.rooms['Room 1']['is_door_locked']:
            self.current_room = 'Room 2'
            return "You enter the next room."
        elif self.current_room == 'Room 2' and not self.rooms['Room 2']['is_door_locked']:
            return self.exit_room()
        else:
            return "The door is locked. You can't enter the next room yet."

    def exit_room(self):
        if self.current_room == 'Room 2':
            self.game_over = True
            self.win = True
            return "You exit the room and win the game!"
        else:
            return "You can't exit from here."

    def drink_potion(self):
        if 'potion' in self.inventory:
            self.inventory.remove('potion')
            return "You drink the potion. You feel stronger!"
        else:
            return "You don't have a potion to drink."

    def info(self):
        state = "Current Room State:\n"
        state += self.look_around()
        state += "\nYour inventory: " + ", ".join(self.inventory) + "\n"
        state += "\nAvailable actions:\n"
        for action in self.actions:
            state += f"- {action}\n"
        return state

    def play(self, action):
        if self.game_over:
            return "Game over. Better luck next time!"

        if action in self.actions:
            return self.actions[action]()
        else:
            return "You can't do that right now."

# Example usage
# escape_room = EscapeRoom()
# print(escape_room.info())
# print(escape_room.play('look around'))
# print(escape_room.play('pick up key'))
# print(escape_room.play('unlock door'))
# print(escape_room.play('enter next room'))
# print(escape_room.play('look around'))
# print(escape_room.play('pick up treasure'))
# print(escape_room.play('exit'))

In [48]:
agent = Agent('Escape Room Solver', 
'''You are an Agent meant to solve the Escape Room Game.
You must output one of the Available Actions at each time step.
If previous action fails, try another.''',
llm = custom_llm)

In [53]:
conversable_agent = ConversableAgent(agent, 
                persistent_memory = {'Inventory': '', 'List of Past Actions': '', 'Summary of Past Actions' : '', 'List of Rooms Explored': ''},  
                person = 'Game')

In [54]:
# To play the game
game = EscapeRoom()
newstate = ''

while not game.game_over:
    state = newstate + '\n' + game.info() + '\nWhat would you do? You must reply with one of the Available actions'
    print('Game:', state)
    reply = conversable_agent.chat(state)
    print(conversable_agent.agent.agent_name, ':', reply)
    print('###\n')
    newstate = game.play(reply)

Game: 
Current Room State:
You look around and see the following items in Room 1:
- key
- potion
There is a locked door here.

Your inventory: 

Available actions:
- look around
- pick up key
- pick up lock
- pick up potion
- pick up treasure
- unlock door
- enter next room
- exit
- drink potion

What would you do? You must reply with one of the Available actions
Thoughts: Since the room contains a key and a potion, we should consider picking them up. If the door is locked, we might need to unlock it using the key. Drinking the potion could also be beneficial. We should reply with one of the available actions based on these considerations.
Persistent Memory: {'Inventory': 'key', 'List of Past Actions': 'pick up key', 'Summary of Past Actions': 'Picked up key in Room 1', 'List of Rooms Explored': 'Room 1'}
Summary of Conversation: The agent decided to pick up the key in the room, indicating progress towards potentially unlocking the door and advancing in the game.
Escape Room Solver : p